# ESSAI CLINIQUE : TABLEAU 1

## Chargement et Définition de la Population (DM)

Créer la population de référence (Population en Intention de Traiter - ITT).

In [1]:
import numpy as np
import pandas as pd

print("BLOC 1 : DÉMOGRAPHIE (DM) ")

# 1. Chargement des deux parties
try:
    dm1 = pd.read_csv("dm1.csv")
    dm2 = pd.read_csv("dm2.csv")
    dm = pd.concat([dm1, dm2], ignore_index=True)
    print(f"Chargement réussi : {len(dm)} patients au total.")
except Exception as e:
    print(f" Erreur de chargement DM : {e}")
    # Fallback si dm.csv existe seul
    # dm = pd.read_csv("dm.csv") 

# 2. Nettoyage de la clé de jointure (CRUCIAL)

# On standardise en Texte + Majuscules + Sans espaces
# Justification Technique : Les espaces invisibles ou les différences de casse (majuscule/minuscule)
# sont la cause n°1 des échecs de fusion (résultats NaN)
dm['USUBJID'] = dm['USUBJID'].astype(str).str.strip().str.upper()

# 3. NETTOYAGE DES VARIABLES DÉMOGRAPHIQUES

# Conversion de l'âge en numérique pour pouvoir calculer la moyenne/écart-type
dm['AGE_clean'] = pd.to_numeric(dm['AGE'], errors='coerce')

# Sexe (Standardisation M/F)
dm['SEX_clean'] = dm['SEX'].astype(str).str.strip().str.upper()
dm['SEX_clean'] = dm['SEX_clean'].replace({'M': 'Male', 'F': 'Female', 'UNK': 'Unknown'})

# Race
dm['RACE_clean'] = dm['RACE'].fillna('Unknown').astype(str).str.strip().str.upper()
dm['RACE_clean'] = dm['RACE_clean'].replace({'NAN': 'Unknown', '': 'Unknown'})

# 4. Création du DataFrame de base (Population)
dm_base = dm[['USUBJID', 'AGE_clean', 'SEX_clean', 'RACE_clean', 'ETHNIC']].rename(columns={
    'AGE_clean': 'AGE', 'SEX_clean': 'SEX', 'RACE_clean': 'RACE'
})

print("Aperçu DM Base :")
display(dm_base.head(3))

BLOC 1 : DÉMOGRAPHIE (DM) 
Chargement réussi : 411 patients au total.
Aperçu DM Base :


,USUBJID,AGE,SEX,RACE,ETHNIC
0,01_000579,27.282683,Female,WHITE,NaN
1,01_001362,41.429158,Female,"BLACK, AFRICAN AMERICAN, OR NEGRO",NaN
2,01_001490,30.392882,Male,WHITE,NaN


## Signes Vitaux et Correction IMC (VS)

Obtenir une mesure unique de poids, taille et IMC par patient

In [2]:
print("\nBLOC 2 : SIGNES VITAUX (VS) ")

# 1. Chargement
try:
    vs1 = pd.read_csv("vs1.csv")
    vs2 = pd.read_csv("vs2.csv")
    vs = pd.concat([vs1, vs2], ignore_index=True)
    print(f" Chargement réussi : {len(vs)} mesures.")
except:
    print(" Fichiers vs1/vs2 absents, tentative avec vs.csv...")
    vs = pd.read_csv("vs.csv")

# 2. Nettoyage Clé
vs['USUBJID'] = vs['USUBJID'].astype(str).str.strip().str.upper()

# 3. Filtrage Baseline & Extraction
vs_bl = vs.copy() 
if 'VSBLFL' in vs.columns:
    vs_bl = vs[vs['VSBLFL'] == 'Y'].copy() # Garder seulement la ligne 'Baseline Flag'

anthro = vs_bl[vs_bl['VSTESTCD'].isin(['HEIGHT', 'WEIGHT'])].copy()
anthro['VSSTRESN_num'] = pd.to_numeric(anthro['VSSTRESN'], errors='coerce')

# 4. Pivot (1 ligne par patient)
anthro_pivot = anthro.pivot_table(index='USUBJID', columns='VSTESTCD', values='VSSTRESN_num', aggfunc='last').reset_index()

# 5. CONVERSION DES UNITÉS (Impérial -> Métrique)
# Hypothèse validée : les données brutes sont en Pouces et Livres
if 'HEIGHT' in anthro_pivot.columns:
    anthro_pivot['HEIGHT_cm'] = anthro_pivot['HEIGHT'] * 2.54  # Pouces -> cm
else:
    anthro_pivot['HEIGHT_cm'] = np.nan 

if 'WEIGHT' in anthro_pivot.columns:
    anthro_pivot['WEIGHT_kg'] = anthro_pivot['WEIGHT'] * 0.453592 # Livres -> kg
else:
    anthro_pivot['WEIGHT_kg'] = np.nan

# 6. Calcul IMC (Poids kg / Taille m²)
anthro_pivot['BMI'] = anthro_pivot['WEIGHT_kg'] / ((anthro_pivot['HEIGHT_cm'] / 100) ** 2)

print("Aperçu Anthro (avec BMI corrigé) :")
display(anthro_pivot[['USUBJID', 'HEIGHT_cm', 'WEIGHT_kg', 'BMI']].head(3))


BLOC 2 : SIGNES VITAUX (VS) 
 Chargement réussi : 18994 mesures.
Aperçu Anthro (avec BMI corrigé) :


VSTESTCD,USUBJID,HEIGHT_cm,WEIGHT_kg,BMI
0,01_000579,170.18,58.966960,20.360653
1,01_001362,147.32,81.192968,37.410628
2,01_001490,182.88,70.306760,21.021546


## Antécédents (MH) et Événements (AE)

Transformer des données textuelles complexes en indicateurs simples (Oui/Non) pour le tableau.

In [3]:
print("\nBLOC 4 : HISTORIQUE (MH) & AEs")

# --- MH (Antécédents) ---
try:
    mh = pd.read_csv("mh.csv") # Souvent un seul fichier pour MH, sinon concaténer
except:
    mh1 = pd.read_csv("mh1.csv")
    mh2 = pd.read_csv("mh2.csv")
    mh = pd.concat([mh1, mh2])

mh['USUBJID'] = mh['USUBJID'].astype(str).str.strip().str.upper()

# Création des indicateurs
# Liste des mots-clés pour regrouper les maladies par système
conditions_interest = {
    'CARDIOVASCULAR': ['CARDIOVASCULAR','HEART','MYOCARDIAL','CORONARY'],
    'RESPIRATORY': ['RESPIRATORY','ASTHMA'],
    'HEPATIC': ['HEPATIC'] # Ajoutez vos autres catégories ici
}

# Concaténation de tous les termes médicaux d'un patient dans une seule cellule
# Gestion de la colonne 'MHTERM' (standard CDISC)
target_col = 'MHTERM' if 'MHTERM' in mh.columns else 'term'
patient_conditions = mh.groupby('USUBJID')[target_col].apply(lambda x: ' || '.join(x.dropna().astype(str).unique())).reset_index()
patient_conditions = patient_conditions.rename(columns={target_col:'MH_ALL'})

# Création des drapeaux (Flags) True/False pour chaque catégorie
for cond, keywords in conditions_interest.items():
    patient_conditions[cond] = patient_conditions["MH_ALL"].str.contains('|'.join(keywords), case=False, na=False)

# 2. PRÉPARATION DES ÉVÉNEMENTS INDÉSIRABLES (AE)

# Comptage du nombre total d'événements par patient
# Détection si au moins un événement grave (Serious AE) est survenu
# Fusion de ces indicateurs
try:
    ae = pd.read_csv("ae.csv")
except:
    ae1 = pd.read_csv("ae1.csv")
    ae2 = pd.read_csv("ae2.csv")
    ae = pd.concat([ae1, ae2])

ae['USUBJID'] = ae['USUBJID'].astype(str).str.strip().str.upper()

ae_counts = ae.groupby('USUBJID').size().reset_index(name='AE_COUNT')
ae_serious = ae.groupby('USUBJID')['AESER'].apply(lambda x: (x == 'Y').any()).reset_index(name='AE_ANY_SERIOUS')
ae_by_subject = ae_counts.merge(ae_serious, on='USUBJID', how='outer')

print("Indicateurs MH et AE prêts.")


BLOC 4 : HISTORIQUE (MH) & AEs
Indicateurs MH et AE prêts.


## Données de Laboratoire (LB)

Extraire les résultats d'analyse biologique (ex: Créatinine) à l'inclusion.

In [4]:
print("\nBLOC 3 : LABORATOIRE (LB)")

# 1. Chargement
try:
    lb1 = pd.read_csv("lb1.csv")
    lb2 = pd.read_csv("lb2.csv")
    lb = pd.concat([lb1, lb2], ignore_index=True)
    print(f" Chargement réussi : {len(lb)} lignes.")
except:
    lb = pd.read_csv("lb.csv")

# 2. Nettoyage Clé
lb['USUBJID'] = lb['USUBJID'].astype(str).str.strip().str.upper()

# 3. Filtrage & Pivot
lb_bl = lb.copy()
if 'LBBLFL' in lb.columns:
    lb_bl = lb[lb['LBBLFL'] == 'Y'].copy()

lb_bl['LBSTRESN_num'] = pd.to_numeric(lb_bl['LBSTRESN'], errors='coerce')
lb_pivot = lb_bl.pivot_table(index='USUBJID', columns='LBTESTCD', values='LBSTRESN_num', aggfunc='last').reset_index()

print("Colonnes Labo disponibles :", lb_pivot.columns.tolist())


BLOC 3 : LABORATOIRE (LB)
 Chargement réussi : 17977 lignes.
Colonnes Labo disponibles : ['USUBJID', 'CREAT']


## Fusion Finale

Left Join : On utilise dm_base comme base gauche. Cela garantit que le tableau final représente bien la population étudiée (ITT)

Formatage : Les standards scientifiques demandent "Moyenne ± Écart-type" pour les chiffres continus et "n (%)" pour les catégories.

In [5]:
print("\nBLOC 5 : FUSION ET TABLEAU FINAL")
import warnings
warnings.filterwarnings('ignore')
pd.set_option('future.no_silent_downcasting', True)
# 1. Fusion (Le 'Left Join' sur dm_base est la clé pour garder la bonne population)
df = dm_base.merge(anthro_pivot[['USUBJID','HEIGHT_cm','WEIGHT_kg','BMI']], on='USUBJID', how='left')
df = df.merge(lb_pivot, on='USUBJID', how='left')
df = df.merge(patient_conditions, on='USUBJID', how='left')
df = df.merge(ae_by_subject, on='USUBJID', how='left')

# 2. Nettoyage post-fusion
if 'AE_COUNT' in df.columns: df['AE_COUNT'] = df['AE_COUNT'].fillna(0).astype(int)
if 'AE_ANY_SERIOUS' in df.columns:
    df['AE_ANY_SERIOUS'] = df['AE_ANY_SERIOUS'].fillna(False).astype(bool)

df_unique = df.drop_duplicates(subset='USUBJID')

# 3. Génération du Tableau
cont_vars = ['AGE', 'HEIGHT_cm', 'WEIGHT_kg', 'BMI', 'AE_COUNT'] + [c for c in lb_pivot.columns if c != 'USUBJID']
cont_vars = [v for v in cont_vars if v in df_unique.columns]

cat_vars = ['SEX', 'RACE', 'ETHNIC'] + list(conditions_interest.keys()) + ['AE_ANY_SERIOUS']
cat_vars = [v for v in cat_vars if v in df_unique.columns]

table_rows = []

# Calculs pour les Variables Continues (Moyenne, Médiane)
for v in cont_vars:
    s = df_unique[v].dropna()
    if not s.empty:
        table_rows.append({
            'Variable': v, 'Modalité': '', 'N': int(s.count()),
            'Mean ± SD': f"{s.mean():.1f} ± {s.std():.1f}",
            'Median [IQR]': f"{s.median():.1f} [{s.quantile(0.25):.1f} - {s.quantile(0.75):.1f}]"
        })

# Calculs pour les Variables Catégorielles (Fréquences)
for v in cat_vars:
    counts = df_unique[v].value_counts(dropna=False)
    total = df_unique.shape[0]
    for level, n in counts.items():
        level_str = str(level) if pd.notna(level) else "Missing"
        pct = (n / total) * 100
        table_rows.append({
            'Variable': v, 'Modalité': level_str, 'N': int(n),
            'Mean ± SD': '', 
            'Median [IQR]': f"{int(n)} ({pct:.1f}%)"
        })

# 4. CRÉATION ET EXPORT DU DATAFRAME FINAL
final_table = pd.DataFrame(table_rows)
final_table.to_csv("Tableau1_Final_Clean.csv", index=False)
print("Tableau généré et sauvegardé !")
display(final_table.head(20))


BLOC 5 : FUSION ET TABLEAU FINAL
Tableau généré et sauvegardé !


,Variable,Modalité,N,Mean ± SD,Median [IQR]
0,AGE,,343,38.0 ± 10.1,39.5 [29.5 - 45.6]
1,HEIGHT_cm,,373,177.3 ± 85.8,172.7 [167.6 - 180.3]
2,WEIGHT_kg,,374,75.5 ± 16.1,73.5 [63.5 - 85.3]
3,BMI,,372,25.1 ± 5.0,24.3 [21.9 - 27.7]
4,AE_COUNT,,411,10.1 ± 12.6,5.0 [0.0 - 16.0]
5,CREAT,,333,148.5 ± 76.0,140.1 [92.5 - 196.0]
6,SEX,Male,281,,281 (68.4%)
7,SEX,Female,128,,128 (31.1%)
8,SEX,U,2,,2 (0.5%)
9,RACE,WHITE,178,,178 (43.3%)
